In [53]:
import utils
import numpy as np

from tqdm import tqdm
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

In [54]:
dji = utils.get_dji()
spy = utils.get_spy()
qqq = utils.get_qqq()

# symbols = set(spy['Symbol'])
symbols = set(dji['Symbol']) | set(spy['Symbol']) | set(qqq['Symbol'])

DJI: Contains 30 tickers.
SPY: Contains 503 tickers.
QQQ: Contains 65 tickers.


In [55]:
args = utils.load_args()
startdate, enddate = utils.get_dates(args)

market_return, riskfree_return, nonholiday_dates = utils.get_benchmark_data(args.benchmark.market, args.benchmark.riskfree, startdate, enddate)
market_data = utils.get_pdr_data(args.benchmark.market, startdate-utils.parse_period_date(args.sharpe.period), enddate, progress=False, clean=False)
stocks_data = utils.get_pdr_data(symbols, startdate-utils.parse_period_date(args.sharpe.period), enddate, progress=True, clean=True)

[*********************100%%**********************]  516 of 516 completed

7 Failed downloads:
['ATVI', 'CTXS', 'FISV', 'CERN', 'BBBY', 'BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2021-09-17 -> 2024-03-17)')


In [56]:
sharpe_update_dates = utils.linspace_datetime(nonholiday_dates, startdate, enddate, delta=utils.parse_period_date(args.sharpe.period))
rebalance_update_dates = utils.linspace_datetime(nonholiday_dates, startdate, enddate, delta=utils.parse_period_date(args.rebalance.period))

In [57]:
portfolio = utils.Portfolio(args.display.initial_balance)
stocks_return = utils.compute_log_return(stocks_data, was_annual=False, retain_symbols=True)

for dt in tqdm(nonholiday_dates, desc="nonholiday_dates"):
    if dt in rebalance_update_dates:
        # Compute sharpe.
        startdate_temp = dt - utils.parse_period_date(args.sharpe.period)
        enddate_temp = dt
        sharpe_df = utils.get_stocks_sharpe_from_data(stocks_data, args.benchmark.riskfree, startdate_temp, enddate_temp)
        
        # Rebalance portfolio.
        market_sharpe = utils.get_stocks_sharpe_from_data(market_data, args.benchmark.riskfree, startdate_temp, enddate_temp)
        portfolio.rebalance(sharpe_df.head(args.params.no_of_companies), min_threshold=market_sharpe[0])

        if args.display.verbose:
            print("Date:", dt)
            print("Market Sharpe:", market_sharpe[0])
            portfolio.display()
            print('-'*80)
    
    # Update portfolio with today's returns.
    portfolio.update(stocks_return, dt)

nonholiday_dates:   0%|          | 0/501 [00:00<?, ?it/s]

nonholiday_dates: 100%|██████████| 501/501 [00:10<00:00, 50.08it/s]


In [58]:
# Final portfolio.
portfolio.display()

        !Utility   %Weight     $Value
Ticker                               
NRG     0.251311  0.115599  14.227186
HIG     0.239296  0.109253  13.446142
ALL     0.231366  0.107393  13.217255
GE      0.213898  0.098811  12.161012
JPM     0.213015  0.098360  12.105570
COR     0.212890  0.097324  11.977980
NVDA    0.209396  0.095389  11.739877
PGR     0.207673  0.096131  11.831274
TRV     0.199541  0.090809  11.176187
CMG     0.198801  0.090932  11.191385


In [59]:
print("Dates:           ", nonholiday_dates[0].date(), "to", nonholiday_dates[-1].date())
print("Portfolio return:", 100*(portfolio.value-args.display.initial_balance)/args.display.initial_balance, "%")

Dates:            2022-03-17 to 2024-03-14
Portfolio return: 23.07386726535495 %


In [60]:
test_data = utils.get_pdr_data(['VTI'], startdate, enddate, progress=False, clean=False)
test_return = utils.compute_log_return(test_data, was_annual=False, retain_symbols=True)

print("Test return:", 100*(test_data.tail(1).values[0] - test_data.head(1).values[0]) / test_data.head(1).values[0] , "%")

Test return: 13.985984646743406 %


In [61]:
# Couple of hings to try:
# - Double check performance when rebalancing.
# - Try Sortino's ratio instead.
# - Try also using Market Cap.

In [62]:
test_data = utils.get_pdr_data(['VTI'], startdate, enddate, progress=False, clean=False)
test_return = utils.compute_log_return(test_data, was_annual=False, retain_symbols=True)

In [63]:
np.prod(test_return+1)

0    1.13986
dtype: float64

In [64]:
np.prod(1+test_data.pct_change().dropna())

1.1398598464674365

In [65]:
test_data.head(1)

Date
2022-03-17    222.580002
Name: Close, dtype: float64

In [66]:
test_data.tail(1)

Date
2024-03-15    253.710007
Name: Close, dtype: float64

In [67]:
(test_data.tail(1).values - test_data.head(1).values) / test_data.head(1).values 

array([0.13985985])